In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
url = 'https://user.wartale.com/index.asp?page=30'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
data = []
span = soup.find('span')
for a_tag in span.find_all('a'):
    data.append({
        'id_type': a_tag.get('id'),
        'category': a_tag.text
    })

df = pd.DataFrame(data)

df['subtypes'] = df['id_type'].apply(lambda x: f'sub{x}')

df

,id_type,category,subtypes
0,type_1,Weapons,subtype_1
1,type_2,Defenses,subtype_2
2,type_3,Accessories,subtype_3


In [7]:
subtypes = df['subtypes'].tolist()
data2 = []
for stype in subtypes:
    a = soup.find('span', id = stype)
    for a_tag in a.find_all('a'):
        data2.append({
            'subtypes': stype,
            'id_subtype': a_tag.get('id'),
            'type': a_tag.text
        })
df2 = pd.DataFrame(data2)    
df2["items"] = df2['id_subtype'].str.replace('sub_', 'items_')
df2    

,subtypes,id_subtype,type,items
0,subtype_1,sub_2,Axes,items_2
1,subtype_1,sub_5,Bows,items_5
2,subtype_1,sub_4,Claws,items_4
3,subtype_1,sub_19,Daggers,items_19
4,subtype_1,sub_21,Fists,items_21
5,subtype_1,sub_7,Hammers,items_7
6,subtype_1,sub_6,Javelins,items_6
7,subtype_1,sub_20,Phantoms,items_20
8,subtype_1,sub_8,Scythes,items_8
9,subtype_1,sub_3,Swords,items_3


In [8]:
Table1 = df2[['items', 'type']]	
Table1	

,items,type
0,items_2,Axes
1,items_5,Bows
2,items_4,Claws
3,items_19,Daggers
4,items_21,Fists
5,items_7,Hammers
6,items_6,Javelins
7,items_20,Phantoms
8,items_8,Scythes
9,items_3,Swords


In [9]:
import sqlite3
conn = sqlite3.connect('ItemsDB.db')
Table1.to_sql('items', conn, if_exists='replace', index=False)
conn.close()



In [11]:
data3 = []
for item in items:
    c = soup.find('div', id=item)
    if c:
        for i in c.find_all('tr'):
            if i.td and i.span:
                data3.append({
                    "items": item,
                    'item_id': i.td['onclick'].split("(")[1].split(",")[0],
                    'item_name': i.span.text
                })



df3 = pd.DataFrame(data3)
df3

,items,item_id,item_name
0,items_2,627,Stone Axe
1,items_2,628,Steel Axe
2,items_2,629,Battle Axe
3,items_2,630,War Axe
4,items_2,631,Double Sided War Axe
...,...,...,...
746,items_18,502,Fury Sheltom
747,items_18,498,Oredo
748,items_18,499,Sapphire
749,items_18,5911,Sol


In [12]:
itemsid = df3['item_id'].tolist()

def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

xd1 = pd.DataFrame()  # Create an empty DataFrame to store the extracted data

for item in itemsid:
    url = f'https://user.wartale.com/guide.asp?page=1&action=2&id={item}'
    response = requests.get(url)
    soupitem = BeautifulSoup(response.content, 'html.parser')
    
    item_box_header = soupitem.find('div', class_='item_box_header')
    item_name = item_box_header.contents[0].strip()
    item_id = soupitem.find("div", class_="item_box_img").img["alt"]
    item_image_url = soupitem.find("div", class_="item_box_img").img["src"]
    
    tr_indices = [1, 10]
    data = {
        "xd": item,
        "item_name": item_name,
        "item_id": item_id,
        "item_image_url": item_image_url
    }
    
    for index in tr_indices:
        try:
            tr = soupitem.find_all('tr')[index]
            data.update(extract_values(tr))
            
            # Extract price and weight values
            price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
            data["weightxd"] = price_weight_values[-2]
            data["pricexd"] = price_weight_values[-1]
        except IndexError:
            pass
    
    df = pd.DataFrame([data])
    xd1 = pd.concat([xd1, df], ignore_index=True)
    time.sleep(2)
    print(item)

495


In [13]:
xd1

,xd,item_name,item_id,item_image_url,Level,Spirit,weightxd,pricexd,Secondary Spec,Weight,Price
0,495,Inferna,OS111,images/items/itOS111.jpg,80,120,2,200000,MS,FS,PS


In [ ]:
df_aux = xd1.copy()
df_aux.drop(['', 'Primary Spec', 'Secondary Spec', 'Weight', 'Price'], axis=1, inplace=True)
df_aux.rename(columns={
                        'weightxd': 'Weight',
                        'pricexd': 'Price'}, inplace=True)

df_aux.fillna(0, inplace=True)
df_aux

In [ ]:
df3.drop(['item_name'], axis=1, inplace=True)
df3

In [ ]:
Table2 = pd.merge(df_aux, df3, on='item_id', how='left')
Table2

In [ ]:
base_url = 'https://user.wartale.com/'
Table2['url'] = Table2['item_image_url'].apply(lambda x: base_url + x)
Table2.drop(['item_image_url'], axis=1, inplace=True)
Table2.rename(columns={'url': 'item_image_url',
                      'title': 'hands' }, inplace=True)
Table2

In [14]:
Table2 = pd.read_csv('Table2.csv')
Table2

,item_id,item_name,game_id,Weight,Price,Level,Strength,Talent,Agility,Spirit,Health,items,item_image_url
0,627,Stone Axe,WA101,5,60,0,0,0,0,0,0,items_2,https://user.wartale.com/images/items/itWA101.jpg
1,628,Steel Axe,WA102,8,420,0,0,0,0,0,0,items_2,https://user.wartale.com/images/items/itWA102.jpg
2,629,Battle Axe,WA103,12,800,7,38,0,0,0,0,items_2,https://user.wartale.com/images/items/itWA103.jpg
3,630,War Axe,WA104,15,1800,10,50,30,0,0,0,items_2,https://user.wartale.com/images/items/itWA104.jpg
4,631,Double Sided War Axe,WA105,19,4600,16,60,36,0,0,0,items_2,https://user.wartale.com/images/items/itWA105.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,502,Fury Sheltom,OS132,5,200000,100,0,0,0,90,0,items_18,https://user.wartale.com/images/items/itOS132.jpg
747,498,Oredo,OS114,2,380000,110,0,0,0,150,0,items_18,https://user.wartale.com/images/items/itOS114.jpg
748,499,Sapphire,OS115,2,440000,115,0,0,0,155,0,items_18,https://user.wartale.com/images/items/itOS115.jpg
749,5911,Sol,OS116,2,500000,120,0,0,0,160,0,items_18,https://user.wartale.com/images/items/itOS116.jpg


In [15]:
import sqlite3
conn = sqlite3.connect('ItemsDB.db')
Table2.to_sql('bitems', conn, if_exists='replace', index=False)
conn.close()

In [16]:
items = ['items_2', 'items_5', 'items_4', 'items_19', 'items_21', 'items_7', 'items_6', 'items_20', 'items_8', 'items_3', 'items_1']
weapondf = Table2[Table2['items'].isin(items)]
weaponlist = weapondf['item_id'].tolist()
weaponlist

[627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 650,
 651,
 647,
 652,
 648,
 653,
 654,
 649,
 655,
 892,
 4909,
 11056,
 17165,
 19225,
 20783,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779,
 780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791,
 792,
 822,
 823,
 793,
 824,
 794,
 825,
 826,
 795,
 827,
 896,
 4911,
 11063,
 17168,
 19230,
 20795,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671,
 672,
 673,
 674,
 675,
 679,
 676,
 680,
 677,
 681,
 682,
 678,
 683,
 893,
 4915,
 11062,
 17167,
 19226,
 20784,
 8992,
 8993,
 8995,
 8994,
 8996,
 8997,
 8998,
 8999,
 9000,
 9001,
 9002,
 9003,
 9004,
 9005,
 9006,
 9007,
 9008,
 9009,
 9010,
 9011,
 9043,
 9012,
 9013,
 9014,
 9015,
 9016,
 11067,
 17172,
 19233,
 20799,
 20333,
 20335,
 20336,
 20337,
 20338,
 20339,
 20340,
 20341,
 20342,
 20343,
 20344,
 20345,
 20346,
 20347,
 20348,
 20

In [ ]:
def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

xd1 = pd.DataFrame()  # Create an empty DataFrame to store the extracted data

for item in weaponlist:
    url = f'https://user.wartale.com/guide.asp?page=1&action=2&id={item}'
    response = requests.get(url)
    soupitem = BeautifulSoup(response.content, 'html.parser')
    
    item_box_header = soupitem.find('div', class_='item_box_header')
    item_name = item_box_header.contents[0].strip()
    
    tr_indices = [4]
    data = {
        "item_id": item,
    }
    
    for index in tr_indices:
        try:
            tr = soupitem.find_all('tr')[index]
            data.update(extract_values(tr))
            # Extract price and weight values
            price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
        except IndexError:
            pass
        
    
    df = pd.DataFrame([data])
    xd1 = pd.concat([xd1, df], ignore_index=True)
    time.sleep(1)
    print(item)




In [ ]:
xd1.to_csv('weapons.csv', index=False)

In [ ]:
def extract_values(tr):
    left_values = tr.select_one('td.item_box_text_left').get_text('\n', strip=True).split('\n')
    right_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
    
    data_dict = {}
    for left, right in zip(left_values, right_values):
        data_dict[left] = right
    return data_dict

xd2 = pd.DataFrame()  # Create an empty DataFrame to store the extracted data


for item in weaponlist:
    url = f'https://user.wartale.com/guide.asp?page=1&action=2&id={item}'
    response = requests.get(url)
    soupitem = BeautifulSoup(response.content, 'html.parser')
    
    item_box_header = soupitem.find('div', class_='item_box_header')
    item_name = item_box_header.contents[0].strip()
    
    tr_indices = [7]
    data = {
        "item_id": item,
    }
    
    for index in tr_indices:
        try:
            tr = soupitem.find_all('tr')[index]
            data.update(extract_values(tr))
            # Extract price and weight values
            price_weight_values = tr.select_one('td:not(.item_box_text_left)').get_text('\n', strip=True).split('\n')
        except IndexError:
            pass
        
    
    df = pd.DataFrame([data])
    xd2 = pd.concat([xd2, df], ignore_index=True)
    time.sleep(1)
    print(item)
xd2    

In [ ]:
xd3 = xd2.copy()

In [ ]:
xd3.rename(columns={col: f"Special_{col}" for col in df.columns }, inplace=True)
xd3.rename(columns={'Special_item_id': 'item_id'}, inplace=True)
xd3

In [ ]:
xd3.rename(columns={'Special_item_id': 'item_id'}, inplace=True)
xd3

In [ ]:
test = xd1.copy()
test[['Min_Min_Damage', 'Max_Min_Damage']] = test['Min Damage'].str.split(' - ', expand=True)
test[['Min_Min_Damage', 'Max_Min_Damage']] = test[['Min_Min_Damage', 'Max_Min_Damage']].apply(pd.to_numeric, errors='coerce')
test

In [ ]:
xd2.to_csv('special_weapons.csv', index=False)

In [2]:
weapondf = pd.read_csv('weapons.csv')
specialweapondf = pd.read_csv('special_weapons.csv')

In [3]:
weapondf

,item_id,Min Damage,Max Damage,Attack Speed,Critical,Attack Rating,Integrity,Block,Evade,Range,Defense,Add HP,Add MP
0,627,1 - 2,3 - 4,6,3%,30 - 38,30 - 36,NaN,NaN,NaN,NaN,NaN,NaN
1,628,2 - 3,4 - 5,6,3%,35 - 43,34 - 45,NaN,NaN,NaN,NaN,NaN,NaN
2,629,3 - 4,6 - 7,5,4%,39 - 45,42 - 55,NaN,NaN,NaN,NaN,NaN,NaN
3,630,4 - 5,8 - 10,7,5%,44 - 56,50 - 65,NaN,NaN,NaN,NaN,NaN,NaN
4,631,7 - 8,16 - 19,6,9%,45 - 60,55 - 70,3 - 3%,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,4913,103 - 107,114 - 118,8,18%,226 - 240,210 - 220,10 - 10%,NaN,210.0,NaN,NaN,NaN
371,11064,112 - 116,123 - 127,8,19%,248 - 266,220 - 230,10 - 10%,NaN,210.0,NaN,NaN,NaN
372,17164,93 - 100,104 - 109,8,18%,268 - 291,225 - 230,NaN,NaN,205.0,NaN,NaN,NaN
373,19228,126 - 130,137 - 141,8,19%,270 - 290,240 - 250,11 - 11%,NaN,210.0,NaN,NaN,NaN


In [4]:
weapondf[['Min_Min_Damage', 'Max_Min_Damage']] = weapondf['Min Damage'].str.split(' - ', expand=True)
weapondf[['Min_Min_Damage', 'Max_Min_Damage']] = weapondf[['Min_Min_Damage', 'Max_Min_Damage']].apply(pd.to_numeric, errors='coerce')

weapondf[['Min_Max_Damage', 'Max_Max_Damage']] = weapondf['Max Damage'].str.split(' - ', expand=True)
weapondf[['Min_Max_Damage', 'Max_Max_Damage']] = weapondf[['Min_Max_Damage', 'Max_Max_Damage']].apply(pd.to_numeric, errors='coerce')


weapondf[['Min_Attack_Ratting', 'Max_Attack_Ratting']] = weapondf['Attack Rating'].str.split(' - ', expand=True)
weapondf[['Min_Attack_Ratting', 'Max_Attack_Ratting']] = weapondf[['Min_Attack_Ratting', 'Max_Attack_Ratting']].apply(pd.to_numeric, errors='coerce')


weapondf[['Min_Integrity', 'Max_Integrity']] = weapondf['Integrity'].str.split(' - ', expand=True)
weapondf[['Min_Integrity', 'Max_Integrity']] = weapondf[['Min_Integrity', 'Max_Integrity']].apply(pd.to_numeric, errors='coerce')


weapondf[['Min_Block', 'Max_Block']] = weapondf['Block'].str.split(' - ', expand=True)
weapondf[['Min_Block', 'Max_Block']] = weapondf[['Min_Block', 'Max_Block']].apply(pd.to_numeric, errors='coerce')


weapondf[['Min_Evade', 'Max_Evade']] = weapondf['Evade'].str.split(' - ', expand=True)
weapondf[['Min_Evade', 'Max_Evade']] = weapondf[['Min_Evade', 'Max_Evade']].apply(pd.to_numeric, errors='coerce')

weapondf

,item_id,Min Damage,Max Damage,Attack Speed,Critical,Attack Rating,Integrity,Block,Evade,Range,...,Min_Max_Damage,Max_Max_Damage,Min_Attack_Ratting,Max_Attack_Ratting,Min_Integrity,Max_Integrity,Min_Block,Max_Block,Min_Evade,Max_Evade
0,627,1 - 2,3 - 4,6,3%,30 - 38,30 - 36,NaN,NaN,NaN,...,3,4,30,38,30,36,NaN,NaN,NaN,NaN
1,628,2 - 3,4 - 5,6,3%,35 - 43,34 - 45,NaN,NaN,NaN,...,4,5,35,43,34,45,NaN,NaN,NaN,NaN
2,629,3 - 4,6 - 7,5,4%,39 - 45,42 - 55,NaN,NaN,NaN,...,6,7,39,45,42,55,NaN,NaN,NaN,NaN
3,630,4 - 5,8 - 10,7,5%,44 - 56,50 - 65,NaN,NaN,NaN,...,8,10,44,56,50,65,NaN,NaN,NaN,NaN
4,631,7 - 8,16 - 19,6,9%,45 - 60,55 - 70,3 - 3%,NaN,NaN,...,16,19,45,60,55,70,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,4913,103 - 107,114 - 118,8,18%,226 - 240,210 - 220,10 - 10%,NaN,210.0,...,114,118,226,240,210,220,10.0,NaN,NaN,NaN
371,11064,112 - 116,123 - 127,8,19%,248 - 266,220 - 230,10 - 10%,NaN,210.0,...,123,127,248,266,220,230,10.0,NaN,NaN,NaN
372,17164,93 - 100,104 - 109,8,18%,268 - 291,225 - 230,NaN,NaN,205.0,...,104,109,268,291,225,230,NaN,NaN,NaN,NaN
373,19228,126 - 130,137 - 141,8,19%,270 - 290,240 - 250,11 - 11%,NaN,210.0,...,137,141,270,290,240,250,11.0,NaN,NaN,NaN


In [8]:
specialweapondf

,item_id,Attack Speed,Attack Power,Critical,Attack Rating,Range,Block,Magic APT,MP Regen,Special Defense
0,627,1.0,Lv/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,628,1.0,Lv/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,629,2.0,Lv/5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,630,NaN,Lv/5,3%,NaN,NaN,NaN,NaN,NaN,NaN
4,631,2.0,NaN,NaN,Lv/1 - 3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
370,4913,NaN,Lv/6,NaN,NaN,NaN,NaN,15 - 18,3.2 - 3.6,NaN
371,11064,NaN,Lv/6,NaN,NaN,NaN,NaN,16 - 19,3.5 - 3.9,NaN
372,17164,NaN,Lv/7,NaN,NaN,NaN,NaN,13 - 18,1.7 - 2,NaN
373,19228,NaN,Lv/6,NaN,Lv/1 - 3,NaN,NaN,17 - 20,3.8 - 4.2,NaN


In [16]:
specialweapondf.rename(columns={
    'Defense': 'Special Defense',
    'Critical': 'Special Critical',
    'Range': 'Special Range',
    'Block': 'Special Block',
    'Attack Speed' : 'Special Attack Speed',
    'Attack Rating' : 'Special Attack Rating',

}, inplace=True)

In [17]:
final_weapondf = pd.merge(weapondf, specialweapondf, on='item_id')
final_weapondf

,item_id,Min Damage,Max Damage,Attack Speed,Critical,Attack Rating,Integrity,Block,Evade,Range,...,Max_Evade,Special Attack Speed,Attack Power,Special Critical,Special Attack Rating,Special Range,Special Block,Magic APT,MP Regen,Special Defense
0,627,1 - 2,3 - 4,6,3%,30 - 38,30 - 36,NaN,NaN,NaN,...,NaN,1.0,Lv/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,628,2 - 3,4 - 5,6,3%,35 - 43,34 - 45,NaN,NaN,NaN,...,NaN,1.0,Lv/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,629,3 - 4,6 - 7,5,4%,39 - 45,42 - 55,NaN,NaN,NaN,...,NaN,2.0,Lv/5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,630,4 - 5,8 - 10,7,5%,44 - 56,50 - 65,NaN,NaN,NaN,...,NaN,NaN,Lv/5,3%,NaN,NaN,NaN,NaN,NaN,NaN
4,631,7 - 8,16 - 19,6,9%,45 - 60,55 - 70,3 - 3%,NaN,NaN,...,NaN,2.0,NaN,NaN,Lv/1 - 3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,4913,103 - 107,114 - 118,8,18%,226 - 240,210 - 220,10 - 10%,NaN,210.0,...,NaN,NaN,Lv/6,NaN,NaN,NaN,NaN,15 - 18,3.2 - 3.6,NaN
371,11064,112 - 116,123 - 127,8,19%,248 - 266,220 - 230,10 - 10%,NaN,210.0,...,NaN,NaN,Lv/6,NaN,NaN,NaN,NaN,16 - 19,3.5 - 3.9,NaN
372,17164,93 - 100,104 - 109,8,18%,268 - 291,225 - 230,NaN,NaN,205.0,...,NaN,NaN,Lv/7,NaN,NaN,NaN,NaN,13 - 18,1.7 - 2,NaN
373,19228,126 - 130,137 - 141,8,19%,270 - 290,240 - 250,11 - 11%,NaN,210.0,...,NaN,NaN,Lv/6,NaN,Lv/1 - 3,NaN,NaN,17 - 20,3.8 - 4.2,NaN


In [18]:
import sqlite3
conn = sqlite3.connect('ItemsDB.db')
final_weapondf.to_sql('weapons', conn, if_exists='replace', index=False)
conn.close()